<a href="https://colab.research.google.com/github/aknip/Streamlit-Gradio/blob/main/myGPTlab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hello World App

This is a Hello World App. Some words about it. Lorem ipsum.

# Setup and Configuration

In [ ]:
# @title Settings

# @markdown Default model
default_model = 'GPT-3.5' # @param ["GPT-3.5", "GPT-4"]

# @markdown Initial Setup Mode for pip install, fetch credentials etc.
initial_setup_mode = True # @param {type:"boolean"}

# @markdown Debug Mode for extensive logging.
debug_mode = True # @param {type:"boolean"}

# @markdown iOS Mode to develop helper functions, no Gradio.
# @markdown Useful for development on iOS, eg. with Carnets App
ios_mode = False # @param {type:"boolean"}

In [ ]:
if ios_mode == False:
  print('Mac')
else:
  print('iOS')

In [ ]:
if initial_setup_mode == True:
  !pip install openai==0.27.7 yt-dlp==2023.7.6 librosa==0.10.0.post2 pickle-mixin==1.0.2 langchain==0.0.225 PyPDF2==3.0.1 PyMuPDF==1.22.5 -q
else:
  print('No initial setup.')


In [ ]:
if (initial_setup_mode == True) and (ios_mode == False) :
  !pip install gradio -q
else:
  print('No initial setup / iOS.')

In [ ]:
if (initial_setup_mode == True) and (ios_mode == False) :
  %load_ext gradio
else:
  print('No initial setup / iOS.')

In [ ]:
if initial_setup_mode == True:
  !wget -q bit.ly/aknip-colab-setup
  %run aknip-colab-setup
else:
  print('No initial setup.')

Secrets (JSON string): ··········
Credentials stored in env 'CREDS'. Get values via: 
creds = json.loads(os.getenv('CREDS')) 
key = creds['OpenAI']['v1']['credential']


In [ ]:
creds = json.loads(os.getenv('CREDS'))
key = creds['OpenAI']['v2']['credential']
# print(key)

# Helper Functions

In [ ]:
# Some Code

# The App

In [66]:
if ios_mode == False:
  import gradio as gr

  theme = gr.themes.Default(
      primary_hue="slate" # , radius_size=gr.themes.Size(radius_sm="3px", radius_xs="2px", radius_xxs="1px")
  )

  css = """
    .gradio-container {max-width: 700px!important}
    .vspacer1 {margin-top: 50px}
  """


  def greet(name):
    result = "HALLO " + name + "!!!"
    return result

  def save(text):
    result = "HALLO " + text + "!!!"
    return result

  def upload_file(my_file):
    full_upload_path = my_file.name
    just_the_filename = os.path.basename(full_upload_path)
    shutil.copyfile(full_upload_path, "./" + just_the_filename)

    f= open("./" + just_the_filename,'r')
    if f.mode == 'r': contents =f.read()
    f.close()

    debug = just_the_filename + "\n"

    if len(contents) == 0:
      debug = debug + "Error: Upload file lengt 0 bytes"
    else:
      debug = debug + "Upload successful"

    return debug


  with gr.Blocks(theme=theme, css=css) as demo:

      gr.Markdown("# ChatGPTLab 2.0", elem_classes="vspacer1")
      gr.Markdown("### Optimizing your work with LLMs.")

      with gr.Tab("Input Text "):
        gr.Markdown("Please enter text")
        file_output = gr.Textbox(label="Text output")
        upload_button = gr.UploadButton("Click to Upload a File", file_types=[".txt",".md"], file_count="single")
        #upload_button = gr.File()
        upload_button.upload(upload_file, upload_button, file_output)


      with gr.Tab("Step 1"):
        gr.Markdown("Please enter all the data which is needed for this application:")
        text_input = gr.Textbox(label="Filename", value="hello")
        text_output = gr.Textbox(label="Text output")
        text_button = gr.Button("Save")
        text_button.click(save, text_input, text_output, api_name="save")

      with gr.Tab("Step 2"):
        gr.Markdown("Please select the optimization:")
        radio = gr.Radio(
          ["by headline", "by paragraph", "by §§§"], label="Text split method"
        )
        name = gr.Textbox(label="Name", placeholder="Enter text...")
        output = gr.Textbox(label="Output Box")
        greet_btn = gr.Button("Start", scale=0)
        greet_btn.click(fn=greet, inputs=name, outputs=output, api_name="greet")

  demo.launch(quiet=True, share=False)

else:
  print('iOS Mode - Nothing to do.')

<IPython.core.display.Javascript object>